In [2]:
import mlflow
from mlflow.tracking import MlflowClient

import mlflow.pyfunc

MLFLOW_TRACKING_URI=os.environ['MLFLOW_TRACKING_URI']
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

model_name = "fake-news-detect-1"
# latest_versions = client.get_latest_versions(name=model_name)
stage = 'Production'
model_uri = f"models:/{model_name}/{stage}"

In [3]:
mlflow.artifacts.download_artifacts(run_id = 'd92d79a7278d4408b2406d82b2b09aab', dst_path='./artifact')

In [15]:
import pickle

In [8]:
model = mlflow.keras.load_model(
    model_uri = model_uri, dst_path='./main'
)

2022-09-01 18:16:39.338697: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-01 18:16:40.176401: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-09-01 18:16:40.275031: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-01 18:16:40.275079: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [10]:
import tensorflow as tf

In [13]:
model = tf.keras.models.load_model('./main/***REMOVED***/model')

In [41]:
from pathlib import Path

In [46]:
path = 'artifact/model|/***REMOVED***/model'

uri_path = Path.cwd().joinpath('main/***REMOVED***/model').as_uri()

In [45]:
uri_path

In [44]:
Path.cwd().joinpath('main/***REMOVED***/model')

In [16]:
import pickle
with open('../artifact/tokenizer.bin', "rb") as f_in:
    tokenizer = pickle.load(f_in)

In [11]:
for mv in client.search_model_versions("name='fake-news-detect-1'"):
    print(mv)

<ModelVersion: creation_timestamp=1662027138594, current_stage='Production', description='', last_updated_timestamp=1662027803213, name='fake-news-detect-1', run_id='d92d79a7278d4408b2406d82b2b09aab', run_link='', source='***REMOVED***/d92d79a7278d4408b2406d82b2b09aab/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='2'>
<ModelVersion: creation_timestamp=1662026816835, current_stage='Production', description='', last_updated_timestamp=1662027839298, name='fake-news-detect-1', run_id='d92d79a7278d4408b2406d82b2b09aab', run_link='', source='***REMOVED***/d92d79a7278d4408b2406d82b2b09aab/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='1'>


In [17]:
from tensorflow.keras.preprocessing import text, sequence
maxlen = 300

In [18]:
tokens = tokenizer.texts_to_sequences("i am a boy")
tokens = sequence.pad_sequences(tokens, maxlen=maxlen)

In [40]:
predicted = model.predict(tokens)

146/146 [==============================] - 27s 187ms/step


In [33]:
predicted

In [32]:
np.where(predicted > 0.5, 1,0)

In [21]:
import numpy as np

In [30]:
classes_x=np.argmax(predicted,axis=1)

In [36]:
loc = '***REMOVED***'

In [38]:
model = mlflow.keras.load_model(f'{loc}/421f9eff7a8a4ce89842c044d9fe6aad/artifacts/model')

In [39]:
model.predict(tokens)

146/146 [==============================] - 28s 184ms/step


In [31]:
classes_x

In [58]:
model = mlflow.keras.load_model(
    model_uri = model_uri, dst_path='./artifact'
)

In [27]:
import pandas as pd
df = pd.read_csv("../***REMOVED***/True.csv")

In [28]:
tokens = tokenizer.texts_to_sequences(df.iloc[0].text)
tokens = sequence.pad_sequences(tokens, maxlen=maxlen)


In [32]:
tokens

In [39]:
import inspect

In [52]:
model._model_impl.predict(tokens)

15/91 [===>..........................] - ETA: 14s

KeyboardInterrupt: 

In [40]:
inspect.getmembers(model)

In [ ]:
class KerasWrapper(mlflow.pyfunc.PythonModel):
  
  def __init__(self, keras_model, labelEncoder, labelDecoder, n):         
    self.keras_model = keras_model
    self.labelEncoder = labelEncoder
    self.labelDecoder = labelDecoder
    self.topn = n
    
  def load_context(self, context): 
    self.keras_model = mlflow.keras.load_model(model_uri=context.artifacts[self.keras_model], compile=False)
  
  def predict(self, context, input_***REMOVED***):
    scores = self.keras_model.predict(input_***REMOVED***)
    return scores